<a href="https://colab.research.google.com/github/AjiSiwi/arunika-temuin/blob/master/Machine%20Learning/stackoverflow_classification_LSTM_3class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
import numpy as np
import pandas as pd

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

Import `matplotlib` and create a helper function to plot graphs:

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

#Load stackoverflow threads


In [ ]:
train_tf = pd.read_csv('train.csv')

In [ ]:
valid_tf = pd.read_csv('valid.csv')

In [ ]:
train_tf.head()

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,HQ


In [ ]:
valid_tf.head()

,Id,Title,Body,Tags,CreationDate,Y
0,34552974,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,<sql><sql-server>,2016-01-01 01:44:52,LQ_EDIT
1,34554721,Retrieve all except some data of the another t...,I have two table m_master and tbl_appointment\...,<php><mysql><sql><codeigniter><mysqli>,2016-01-01 08:43:50,LQ_EDIT
2,34555135,Pandas: read_html,<p>I'm trying to extract US states from wiki U...,<python><pandas>,2016-01-01 09:55:22,HQ
3,34555448,Reader Always gimme NULL,"I'm so new to C#, I wanna make an application ...",<sql-server><c#-4.0>,2016-01-01 10:43:45,LQ_EDIT
4,34555752,php rearrange array elements based on condition,basically i have this array:\r\n\r\n array(...,<php>,2016-01-01 11:34:09,LQ_EDIT


In [ ]:
valid_tf = valid_tf.drop(labels='Id',axis=1)
valid_tf = valid_tf.drop(labels='Tags',axis=1)
valid_tf = valid_tf.drop(labels='CreationDate',axis=1)
valid_tf.head()

,Title,Body,Y
0,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,LQ_EDIT
1,Retrieve all except some data of the another t...,I have two table m_master and tbl_appointment\...,LQ_EDIT
2,Pandas: read_html,<p>I'm trying to extract US states from wiki U...,HQ
3,Reader Always gimme NULL,"I'm so new to C#, I wanna make an application ...",LQ_EDIT
4,php rearrange array elements based on condition,basically i have this array:\r\n\r\n array(...,LQ_EDIT


In [ ]:
train_tf = train_tf.drop(labels='Id',axis=1)
train_tf = train_tf.drop(labels='Tags',axis=1)
train_tf = train_tf.drop(labels='CreationDate',axis=1)
train_tf.head()

,Title,Body,Y
0,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,LQ_CLOSE
1,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,HQ
2,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,HQ
3,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",HQ
4,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,HQ


In [ ]:
print(len(train_tf))
print(len(valid_tf))

45000
15000


#Preprocessing text data

In [ ]:
import re
import string

In [ ]:
def cleaning_text(text):
  '''
  input: one column dataframe
  '''
  text = text.lower()
  text = text.strip()
  text = re.sub(r'\d+', '', text) #remove number character
  text = re.sub(r"\s+", " ", text)
  text = re.sub(r"\w+…|…", "", text) # Remove ellipsis (and last word)
  text = re.sub(r"(?<=\w)-(?=\w)", " ", text)# Replace dash between words
  text = re.sub(f"[{re.escape(string.punctuation)}]", "", text) # Remove punctuation

  return text


In [ ]:
def remove_stopword(text):
  '''a function for removing the stopword'''
  sw = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
       'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his',
       'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself',
       'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which',
       'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
       'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
       'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and',
       'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at',
       'by', 'for', 'with', 'about', 'against', 'between', 'into',
       'through', 'during', 'before', 'after', 'above', 'below', 'to',
       'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under',
       'again', 'further', 'then', 'once', 'here', 'there', 'when',
       'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
       'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own',
       'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will',
       'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've',
       'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn',
       'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn',
       'wasn', 'weren', 'won', 'wouldn']
  # removing the stop words and lowercasing the selected words
  text = [word for word in text.split() if word.lower() not in sw]
  # joining the list of words with space separator
  return " ".join(text)

##One-Hot Label

In [ ]:
# Get one hot encoding of columns B
one_hot_labels_train = pd.get_dummies(train_tf['Y'])
# Drop column B as it is now encoded
train_tf = train_tf.drop('Y',axis = 1)
# Join the encoded df
train_tf = train_tf.join(one_hot_labels_train)
train_tf.head() 

,Title,Body,HQ,LQ_CLOSE,LQ_EDIT
0,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,0,1,0
1,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,1,0,0
2,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,1,0,0
3,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",1,0,0
4,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,1,0,0


In [ ]:
# Get one hot encoding of columns B
one_hot_labels_valid = pd.get_dummies(valid_tf['Y'])
# Drop column B as it is now encoded
valid_tf = valid_tf.drop('Y',axis = 1)
# Join the encoded df
valid_tf = valid_tf.join(one_hot_labels_valid)
valid_tf.head()

,Title,Body,HQ,LQ_CLOSE,LQ_EDIT
0,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,0,0,1
1,Retrieve all except some data of the another t...,I have two table m_master and tbl_appointment\...,0,0,1
2,Pandas: read_html,<p>I'm trying to extract US states from wiki U...,1,0,0
3,Reader Always gimme NULL,"I'm so new to C#, I wanna make an application ...",0,0,1
4,php rearrange array elements based on condition,basically i have this array:\r\n\r\n array(...,0,0,1


## join 'Body' and 'Title'

In [ ]:
train_tf["text"] = train_tf["Title"] + ' ' + train_tf["Body"]

cols = train_tf.columns.tolist()
cols = cols[-1:] + cols[:-1]
train_tf = train_tf[cols] 

train_tf = train_tf.drop(labels='Title',axis=1)
train_tf = train_tf.drop(labels='Body',axis=1)

train_tf.head()

,text,HQ,LQ_CLOSE,LQ_EDIT
0,Java: Repeat Task Every Random Seconds <p>I'm ...,0,1,0
1,Why are Java Optionals immutable? <p>I'd like ...,1,0,0
2,Text Overlay Image with Darkened Opacity React...,1,0,0
3,Why ternary operator in swift is so picky? <p>...,1,0,0
4,hide/show fab with scale animation <p>I'm usin...,1,0,0


In [ ]:
valid_tf["text"] = valid_tf["Title"] + ' ' + valid_tf["Body"]

cols = valid_tf.columns.tolist()
cols = cols[-1:] + cols[:-1]
valid_tf = valid_tf[cols] 

valid_tf = valid_tf.drop(labels='Title',axis=1)
valid_tf = valid_tf.drop(labels='Body',axis=1)

valid_tf.head()

,text,HQ,LQ_CLOSE,LQ_EDIT
0,How to get all the child records from differen...,0,0,1
1,Retrieve all except some data of the another t...,0,0,1
2,Pandas: read_html <p>I'm trying to extract US ...,1,0,0
3,"Reader Always gimme NULL I'm so new to C#, I w...",0,0,1
4,php rearrange array elements based on conditio...,0,0,1


In [ ]:
train_tf['text'] = train_tf['text'].apply(cleaning_text)
valid_tf['text'] = valid_tf['text'].apply(cleaning_text)

In [ ]:
train_tf['text'] = train_tf['text'].apply(remove_stopword)
valid_tf['text'] = valid_tf['text'].apply(remove_stopword)

In [ ]:
train_tf.head()

,text,HQ,LQ_CLOSE,LQ_EDIT
0,java repeat task every random seconds pim alre...,0,1,0
1,java optionals immutable pid like understand j...,1,0,0
2,text overlay image darkened opacity react nati...,1,0,0
3,ternary operator swift picky pthe question sim...,1,0,0
4,hideshow fab scale animation pim using custom ...,1,0,0


In [ ]:
valid_tf.head()

,text,HQ,LQ_CLOSE,LQ_EDIT
0,get child records different tables based given...,0,0,1
1,retrieve except data another table two table m...,0,0,1
2,pandas readhtml pim trying extract us states w...,1,0,0
3,reader always gimme null im new c wanna make a...,0,0,1
4,php rearrange array elements based condition b...,0,0,1


##tokenize text

In [ ]:
VOCAB_SIZE = 10000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)

texts = np.array(train_tf['text']) 

encoder.adapt(texts)

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:200]

array(['', '[UNK]', 'codepre', 'pi', 'p', 'code', 'new', 'using',
       'string', 'int', 'error', 'return', 'want', 'like', 'get', 'data',
       'file', 'public', 'use', 'gt', 'class', 'function', 'c', 'value',
       'im', 'name', 'import', 'void', 'would', 'need', 'one', 'help',
       'array', 'x', 'trying', 'way', 'var', 'else', 'line', 'know',
       'image', 'type', 'id', 'app', 'user', 'list', 'number', 'set',
       'div', 'null', 'text', 'create', 'b', 'following', 'make', 'run',
       'input', 'please', 'find', 'time', 'true', 'python', 'tried',
       'method', 'dont', 'table', 'work', 'private', 'precode', 'object',
       'false', 'first', 'problem', 'add', 'two', 'example', 'button',
       'output', 'n', 'android', 'working', 'values', 'end', 'test',
       'pim', 'static', 'html', 'try', 'result', 'server', 'cant',
       'print', 'main', 'also', 'php', 'something', 'blockquote', 'java',
       'project', 'date', 'case', 'could', 'pthe', 'version', 'getting',
       

In [ ]:
encoded_train= encoder(train_tf['text']).numpy()
encoded_train

array([[  97, 1242,  418, ...,    0,    0,    0],
       [  97,    1, 3966, ...,    0,    0,    0],
       [  50, 2980,   40, ...,    0,    0,    0],
       ...,
       [ 116, 2660,   99, ...,    0,    0,    0],
       [ 447, 1057, 1293, ...,    0,    0,    0],
       [  54,   22,  107, ...,    0,    0,    0]])

In [ ]:
encoded_valid= encoder(valid_tf['text']).numpy()
encoded_valid

array([[  14,  497,  891, ...,    0,    0,    0],
       [ 966,  846,   15, ...,    0,    0,    0],
       [ 791,    1,   84, ...,    0,    0,    0],
       ...,
       [ 958,  568,   10, ...,    0,    0,    0],
       [  94,  104,  205, ...,    0,    0,    0],
       [  90, 5964,  349, ...,    0,    0,    0]])

In [ ]:
len(encoded_valid[0])

2418

# Klasifikasi

##load pretrained embedding layers

In [ ]:
import tensorflow_hub as hub
embedding = ("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1")

hub_layer = hub.KerasLayer(embedding, input_shape=[], output_shape=64, dtype=tf.string, trainable=True)

In [ ]:
model = tf.keras.Sequential([
    # encoder,
    # tf.keras.layers.Embedding(
    #    input_dim=len(encoder.get_vocabulary()),
    #    output_dim=64,
        # Use masking to handle the variable sequence lengths
    #    mask_zero=True),
    hub_layer,
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, 0)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3)
])

In [ ]:
#PAKAI INI KALAU GA PAKE PRETRAINED MODEL
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
#    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3)
])

Compile the Keras model to configure the training process:

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

## Train the model

In [ ]:
#jika tidak pakai pretrained embbedeing layers
history = model.fit(encoded_train, one_hot_labels_train, verbose=1, epochs=30)

Epoch 1/30
1407/1407 [==============================] - 126s 78ms/step - loss: 1.4494 - accuracy: 0.3328
Epoch 2/30
1407/1407 [==============================] - 110s 78ms/step - loss: 1.2188 - accuracy: 0.3996
Epoch 3/30
1407/1407 [==============================] - 110s 78ms/step - loss: 2.2815 - accuracy: 0.2058
Epoch 4/30
1407/1407 [==============================] - 111s 79ms/step - loss: 3.0866 - accuracy: 0.3262
Epoch 5/30
1407/1407 [==============================] - 111s 79ms/step - loss: 1.9095 - accuracy: 0.3695
Epoch 6/30
1407/1407 [==============================] - 110s 79ms/step - loss: 2.0813 - accuracy: 0.2819
Epoch 7/30
1407/1407 [==============================] - 111s 79ms/step - loss: 2.5130 - accuracy: 0.3709
Epoch 8/30
1407/1407 [==============================] - 110s 78ms/step - loss: 1.8376 - accuracy: 0.2020
Epoch 9/30
1407/1407 [==============================] - 111s 79ms/step - loss: 1.5525 - accuracy: 0.3174
Epoch 10/30
1407/1407 [==============================] 

In [ ]:
#uncomment jika tidak pakai pretrained model
test_loss, test_acc = model.evaluate(encoded_valid, one_hot_labels_valid)


print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
#uncomment jika pakai pretrained model
test_loss, test_acc = model.evaluate(valid_tf['text'].values.ravel(), one_hot_labels_valid)

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

Run a prediction on a new sentence:

If the prediction is >= 0.0, it is positive else it is negative.

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))